In [1]:
import json 
import requests
from pprint import pprint

In [2]:
token = 'AQAEA7qhmZa4AAfQkYPU6RXAqERZnf9enrchyyI'
headers = {'Authorization': 'OAuth ' + token}

In [8]:
'''
params = {'metrics': 'ym:s:visits,ym:s:goal137595631reaches',
          'dimensions': 'ym:s:lastsignTrafficSource',  
          'date1': '2020-09-01',  
          'date2': '2020-09-30',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
'''
params = {'metrics': 'ym:s:visits,ym:s:goal137595631reaches',
          'dimensions': 'ym:s:lastsignTrafficSource',  
          'date1': '2020-10-07',  
          'date2': '2020-10-09',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}

In [9]:
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code)

200


In [10]:
metrika_data = response.json()
print(metrika_data.keys())

dict_keys(['query', 'data', 'total_rows', 'total_rows_rounded', 'sampled', 'contains_sensitive_data', 'sample_share', 'sample_size', 'sample_space', 'data_lag', 'totals', 'min', 'max'])


In [11]:
pprint(metrika_data['data'][0:5])

[{'dimensions': [{'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'}],
  'metrics': [345.0, 10.0]},
 {'dimensions': [{'icon_id': '0',
                  'icon_type': 'traffic-source',
                  'id': 'direct',
                  'name': 'Direct traffic'}],
  'metrics': [113.0, 5.0]},
 {'dimensions': [{'icon_id': '1',
                  'icon_type': 'traffic-source',
                  'id': 'referral',
                  'name': 'Link traffic'}],
  'metrics': [16.0, 1.0]},
 {'dimensions': [{'icon_id': '8',
                  'icon_type': 'traffic-source',
                  'id': 'social',
                  'name': 'Social network traffic'}],
  'metrics': [12.0, 1.0]},
 {'dimensions': [{'icon_id': '-1',
                  'icon_type': 'traffic-source',
                  'id': 'internal',
                  'name': 'Internal traffic'}],
  'metrics': [2.0, 0.0]}]


Импортируем библиотеку Pandas и создаём датафрейм metrika_df из данных статистики metrika_data['data']:

In [12]:
import pandas as pd 

metrika_df = pd.DataFrame(metrika_data['data'])
print(metrika_df.head(10)) 

                                          dimensions        metrics
0  [{'icon_id': '2', 'icon_type': 'traffic-source...  [345.0, 10.0]
1  [{'icon_id': '0', 'icon_type': 'traffic-source...   [113.0, 5.0]
2  [{'icon_id': '1', 'icon_type': 'traffic-source...    [16.0, 1.0]
3  [{'icon_id': '8', 'icon_type': 'traffic-source...    [12.0, 1.0]
4  [{'icon_id': '-1', 'icon_type': 'traffic-sourc...     [2.0, 0.0]


Избавляемся от вложенных словарей

In [13]:
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)
pprint(metrika_list_of_dicts[0:5])

[{'ym:s:goal137595631reaches': 10.0,
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:visits': 345.0},
 {'ym:s:goal137595631reaches': 5.0,
  'ym:s:lastsignTrafficSource': 'Direct traffic',
  'ym:s:visits': 113.0},
 {'ym:s:goal137595631reaches': 1.0,
  'ym:s:lastsignTrafficSource': 'Link traffic',
  'ym:s:visits': 16.0},
 {'ym:s:goal137595631reaches': 1.0,
  'ym:s:lastsignTrafficSource': 'Social network traffic',
  'ym:s:visits': 12.0},
 {'ym:s:goal137595631reaches': 0.0,
  'ym:s:lastsignTrafficSource': 'Internal traffic',
  'ym:s:visits': 2.0}]


Создаём датафрейм

In [15]:
metrika_df = pd.DataFrame(metrika_list_of_dicts)
metrika_df.head(10)

,ym:s:lastsignTrafficSource,ym:s:visits,ym:s:goal137595631reaches
0,Search engine traffic,345.0,10.0
1,Direct traffic,113.0,5.0
2,Link traffic,16.0,1.0
3,Social network traffic,12.0,1.0
4,Internal traffic,2.0,0.0


In [16]:
metrika_df.columns=['Source','Visits','Conversions']
metrika_df['Last non-direct CR%'] = metrika_df['Conversions']/metrika_df['Visits']
metrika_df.head(10)

,Source,Visits,Conversions,Last non-direct CR%
0,Search engine traffic,345.0,10.0,0.028986
1,Direct traffic,113.0,5.0,0.044248
2,Link traffic,16.0,1.0,0.062500
3,Social network traffic,12.0,1.0,0.083333
4,Internal traffic,2.0,0.0,0.000000


In [17]:
# какая конверсия (в процентах) будет у Direct traffic в случае применения модели «Последний источник»?

params2 = {'metrics': 'ym:s:visits,ym:s:goal137595631reaches',
          'dimensions': 'ym:s:lastTrafficSource',  
          'date1': '2020-10-07',  
          'date2': '2020-10-09',    
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params2, headers=headers)
print (response.status_code)
metrika_data2 = response.json()
metrika_df2 = pd.DataFrame(metrika_data2['data'])
metrika_list_of_dicts2 = getMetrikaDataInListOfDicts(metrika_data2)
metrika_df2 = pd.DataFrame(metrika_list_of_dicts2)
metrika_df2.columns=['Source','Visits','Conversions']
metrika_df2['Last click CR%'] = metrika_df2['Conversions']/metrika_df2['Visits']
metrika_df2.head(10)

200


,Source,Visits,Conversions,Last click CR%
0,Search engine traffic,317.0,10.0,0.031546
1,Direct traffic,144.0,6.0,0.041667
2,Link traffic,15.0,1.0,0.066667
3,Social network traffic,10.0,0.0,0.000000
4,Internal traffic,2.0,0.0,0.000000


In [18]:
# К какому каналу была отнесена конверсия из источника Direct в модели Last non-direct?

df1 = metrika_df.loc[:,['Source','Last non-direct CR%']]
df2 = metrika_df2.loc[:,['Source','Last click CR%']]
attribution_compare_df = df1.merge(df2, on='Source')
attribution_compare_df['Last click to Last non-direct Difference'] = attribution_compare_df['Last click CR%']/attribution_compare_df['Last non-direct CR%'] -1
attribution_compare_df

,Source,Last non-direct CR%,Last click CR%,Last click to Last non-direct Difference
0,Search engine traffic,0.028986,0.031546,0.088328
1,Direct traffic,0.044248,0.041667,-0.058333
2,Link traffic,0.062500,0.066667,0.066667
3,Social network traffic,0.083333,0.000000,-1.000000
4,Internal traffic,0.000000,0.000000,NaN
